In [34]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from docx import Document
from docx.shared import Pt  # For font size
from docx.oxml.ns import qn  # For language customization
from docx.oxml import parse_xml  # For custom XML styling
from docx.shared import RGBColor  # For font color
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.oxml import OxmlElement
import random
import string
from sentence_transformers import SentenceTransformer, util
import os



In [35]:
chrome_options =  webdriver.ChromeOptions()
title_search_news_list=[]
chrome_options.add_argument("--disable-gpu")  
chrome_options.add_argument("--blink-settings=imagesEnabled=false")  
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2,}
)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
chrome_options.add_argument("--mute-audio")
chrome_options.add_argument("--autoplay-policy=no-user-gesture-required")# Abort the request if it's a video file

doc = Document()
sentence=''

In [36]:
service = Service("C:/Users/mehdi/Documents/webdrivers/chromedriver.exe")
driver = webdriver.Chrome(service=service,options=chrome_options)  

In [37]:
driver.get('https://www.google.com/advanced_search')

In [38]:
current_sentence=''
news_websites_url=['https://akharinkhabar.ir',#0
                   'https://nournews.ir',#1
                   'https://isna.ir',#2
                   'https://farsnews.ir',#3
                   'https://www.irna.ir',#4
                   'https://www.iribnews.ir',#5
                   'https://www.mehrnews.com',#6
                   'https://www.tasnimnews.com'#7
                   ]
key_word = ["لبنان", "غزه", "ایران"]
result_map={}
custom_key=0

In [39]:
public_url=news_websites_url[1]#choose webisite

In [40]:
def compare_texts(text1, text2):
    try:
        model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        embedding1 = model.encode(text1)
        embedding2 = model.encode(text2)
        similarity = util.cos_sim(embedding1, embedding2)
        similarity_percentage = int(similarity.item() *100)
        return similarity_percentage
    except Exception as e:
        return str(e)

# Example usage
text1 = "گردشگری یکی از بهترین راه‌ها برای آشنایی با فرهنگ‌ها و زیبایی‌های طبیعی هر کشور است. طبیعت زیبا و چشم‌اندازهای بی‌نظیر، مانند جنگل‌ها، کوه‌ها و رودخانه‌ها، همیشه گردشگران زیادی را جذب می‌کند. افراد علاقه‌مند به طبیعت می‌توانند با سفر به مناطق مختلف، تجربه‌های منحصربه‌فردی کسب کنند و از آرامش و سکوت دلپذیر طبیعت لذت ببرند."
text2 = "تغذیه مناسب نقش مهمی در حفظ سلامتی دارد. مصرف غذاهای طبیعی مانند میوه‌ها و سبزیجات، به تأمین ویتامین‌ها و مواد معدنی موردنیاز بدن کمک می‌کند. علاوه بر این، تغذیه سالم می‌تواند از بروز بسیاری از بیماری‌ها جلوگیری کند. افراد باید به تعادل در مصرف مواد غذایی و کاهش مصرف چربی‌ها و قندهای مصنوعی توجه ویژه‌ای داشته باشند."

similarity_percentage = compare_texts(text1, text2)
print(similarity_percentage)


25


In [41]:
def compare(key,text):
    # print('----------------------------')
    # print(f'sentence{sentence}')
    # total_count = sum(text.lower().split().count(word.lower()) for word in key_word)
    # # Create the result list
    # result = [total_count, text]
    # print(f'compare customkey{key}')
    # Add the result to a map with a custom key
     # Define your custom key
    # result_map[''.join(random.choices(string.ascii_letters + string.digits, k=10))]=result
    if not result_map:
        result_map[key]=text
    else:
        for k,v in result_map.items():
            if compare_texts(v,text)>65:
                break
            else:
                result_map[key]=text
    # Output
    print("Result:", result)
    print("Map:", result_map)

In [42]:
def save_sentences_to_files(result_map, folder_path="output_files"):
    

    # Create the folder if it doesn't exist
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Iterate over the result_map and save each sentence to a file
    for idx, sentence in enumerate(result_map.values()):
        file_name = os.path.join(folder_path, f"{idx}.txt")
        with open(file_name, "w", encoding="utf-8") as file:
            file.write(sentence)
        print(f"Saved file: {file_name}")






In [43]:
def akharinKhabarnews():
    print('akharin')
    sentence=driver.find_element(By.CLASS_NAME, "content_content__sfzd5").text
    return sentence
    print(sentence)
    print('akharin done!')
    

In [44]:
def tasnim():
    sentence=str(driver.find_element(By.CLASS_NAME, "story"))
    return sentence
    print('tasni done!')

In [45]:
def iribnews():
    container= str(driver.find_element(By.CLASS_NAME, "div.news_center_body"))
    dive= str(driver.find_element(By.CLASS_NAME, "div.body.body_media_content_show"))
    sentence=f"{container} \n {dive}"
    return sentence
    print('iribnews done!')

In [46]:
def noornews():
    sentence= driver.find_element(By.XPATH, '//*[@id="div_balance_right"]/section/div').text
    return sentence
    print('noor done!')

In [47]:
def irna():
    elements = driver.find_elements(By.CSS_SELECTOR, ".item-text p")
    sentence=" ".join([element.text for element in elements])
    return sentence
    print('irna done!')

In [48]:
def fars():
    news_text_elements=driver.find_element(By.CSS_SELECTOR, ".n-nqnd1g span.auto-dir-block")
    sentence=" ".join([element.text for element in news_text_elements])
    return sentence
    print('fars done!')

In [49]:
def mehrnews():
    sentence=driver.find_element(By.CSS_SELECTOR, 'div.item-body div.item-text').text
    return sentence
    print('mehr done!')
    

In [50]:
def isna():
    summary = driver.find_element(By.CSS_SELECTOR, ".summary").text
    article_body = driver.find_element(By.CSS_SELECTOR, ".item-text").text
    sentence=f"{summary}\n\n{article_body}"
    return sentence
    print(' isna done!')

In [51]:
# Fill the 'all these words' field
all_words = driver.find_element(By.ID, 'xX4UFf')
all_words.send_keys('سوریه لبنان')

# Fill the 'this exact word or phrase' field
exact_phrase = driver.find_element(By.ID, 'CwYCWc')
exact_phrase.send_keys('صلح')

# Fill the 'any of these words' field
any_words = driver.find_element(By.ID, 'mSoczb')
any_words.send_keys(' تهران')

# Fill the 'none of these words' field
none_words = driver.find_element(By.ID, 't2dX1c')
none_words.send_keys('نتانیاهو')

# Fill the 'numbers ranging from' field (start and end)
# range_start = driver.find_element(By.ID, 'LK5akc')
# range_start.send_keys('10')

# range_end = driver.find_element(By.NAME, 'as_nhi')  # Note: 'as_nhi' doesn't have an ID but has a name attribute
# range_end.send_keys('35 kg')
driver.find_element(By.ID, 'NqEkZd').send_keys(public_url.replace("https://", "").replace("http://", ""))  # Fill 'Website or domain'
advance_button=driver.find_element(By.XPATH,'//*[@id="s1zaZb"]/div[5]/div[8]/div[2]/input[2]')
advance_button.click()
custom_key=0

In [52]:

# Locate the table containing the page links
table = driver.find_element(By.CLASS_NAME, "AaVjTc")
# Find all anchor tags (<a>) in the table
anchors = table.find_elements(By.TAG_NAME, "a")

# Extract the URLs from the href attribute and store them in a list
urls = [anchor.get_attribute("href") for anchor in anchors]

# Print the first 10 URLs
print(urls[:10])

['https://www.google.com/search?q=%D8%B3%D9%88%D8%B1%DB%8C%D9%87+%D9%84%D8%A8%D9%86%D8%A7%D9%86+%D8%AA%D9%87%D8%B1%D8%A7%D9%86+%22%D8%B5%D9%84%D8%AD%22+-%D9%86%D8%AA%D8%A7%D9%86%DB%8C%D8%A7%D9%87%D9%88+site:nournews.ir&lr=&sca_esv=442be0e50c9187c8&as_qdr=all&ei=NBt6Z5rFKeOkkdUPoMasoA0&start=10&sa=N&sstk=ATObxK6HVi6Jh5n6aVI_IhAEFntHMMJSZ4FXmMwl3vMY_MmkSjkOvt-P_Ktv8cNg0AzpVusCDoI5kZ0PLvpsEcTw6y2VpGr6Xkmirw&ved=2ahUKEwiar4KN792KAxVjUqQEHSAjC9QQ8tMDegQIKhAE', 'https://www.google.com/search?q=%D8%B3%D9%88%D8%B1%DB%8C%D9%87+%D9%84%D8%A8%D9%86%D8%A7%D9%86+%D8%AA%D9%87%D8%B1%D8%A7%D9%86+%22%D8%B5%D9%84%D8%AD%22+-%D9%86%D8%AA%D8%A7%D9%86%DB%8C%D8%A7%D9%87%D9%88+site:nournews.ir&lr=&sca_esv=442be0e50c9187c8&as_qdr=all&ei=NBt6Z5rFKeOkkdUPoMasoA0&start=20&sa=N&sstk=ATObxK6HVi6Jh5n6aVI_IhAEFntHMMJSZ4FXmMwl3vMY_MmkSjkOvt-P_Ktv8cNg0AzpVusCDoI5kZ0PLvpsEcTw6y2VpGr6Xkmirw&ved=2ahUKEwiar4KN792KAxVjUqQEHSAjC9QQ8tMDegQIKhAG', 'https://www.google.com/search?q=%D8%B3%D9%88%D8%B1%DB%8C%D9%87+%D9%84%D8%A8%D9%8

In [53]:
data = []
news_current_urls = []  # Variable to store all URLs
elements = driver.find_elements(By.CLASS_NAME, "LC20lb")  # Locate elements with the specific class
for element in elements:
    title = element.text  # Get the title text
    try:
        parent = element.find_element(By.XPATH, "..")  # Move to the parent element (usually <a>)
        url = parent.get_attribute("href")  # Get the link
        if title and url:
            data.append({"title": title, "url": url})
            news_current_urls.append(url)  # Save the URL in the variable
    except Exception as e:
        print(f"Error processing element: {e}")

# Print Titles and URLs
print("Extracted Titles and URLs:")
for item in data:
    print(f"Title: {item['title']}")
    title_search_news_list.append(item['title'])
    
    print(f"URL: {item['url']}")
    print("-" * 50)

# Print URLs stored in `news_current_urls`
print("All URLs:")
print(news_current_urls)

Extracted Titles and URLs:
Title: درگیری ارتش لبنان با گروه های مسلح سوری
URL: https://nournews.ir/fa/news/206863/%D8%AF%D8%B1%DA%AF%DB%8C%D8%B1%DB%8C-%D8%A7%D8%B1%D8%AA%D8%B4-%D9%84%D8%A8%D9%86%D8%A7%D9%86-%D8%A8%D8%A7-%DA%AF%D8%B1%D9%88%D9%87-%D9%87%D8%A7%DB%8C-%D9%85%D8%B3%D9%84%D8%AD-%D8%B3%D9%88%D8%B1%DB%8C
--------------------------------------------------
Title: وظیفه همه کشورهای صلح دوست، تلاش برای برقراری آتش‌بس ...
URL: https://nournews.ir/fa/news/199962/%D9%88%D8%B8%DB%8C%D9%81%D9%87-%D9%87%D9%85%D9%87-%DA%A9%D8%B4%D9%88%D8%B1%D9%87%D8%A7%DB%8C-%D8%B5%D9%84%D8%AD-%D8%AF%D9%88%D8%B3%D8%AA%D8%8C-%D8%AA%D9%84%D8%A7%D8%B4-%D8%A8%D8%B1%D8%A7%DB%8C-%D8%A8%D8%B1%D9%82%D8%B1%D8%A7%D8%B1%DB%8C-%D8%A2%D8%AA%D8%B4%E2%80%8C%D8%A8%D8%B3-%D8%AF%D8%B1-%D9%84%D8%A8%D9%86%D8%A7%D9%86-%D9%88-%D8%BA%D8%B2%D9%87-%D8%A7%D8%B3%D8%AA
--------------------------------------------------
Title: واکنش وزارت خارجه به برخی ادعاها درباره نحوه خروج ایرانیان ...
URL: https://nournews.ir/fa/news/204738/%D9%8

In [54]:
for index, url in enumerate(news_current_urls):
    print(f"Loading website {index + 1}/{len(news_current_urls)}: {url}")
    driver.get(url)  # Load the URL
    custom_key=index
    
        # Wait until the page has fully loaded (specific condition can be customized)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))  # Wait for the body tag to load
    )
        
    print(f'public:{public_url}')
    try:
        match public_url:
            case   'https://nournews.ir': compare(custom_key,noornews())
            case   'https://isna.ir': compare(custom_key,isna())
            case   'https://farsnews.ir': compare(custom_key,fars())
            case   'https://www.irna.ir': compare(custom_key,irna())
            case   'https://www.iribnews.ir': compare(custom_key,iribnews())
            case   'https://www.mehrnews.com': compare(custom_key,mehrnews())
            case   'https://www.tasnimnews.com': compare(custom_key,tasnim()) 
            case   'https://akharinkhabar.ir':compare(custom_key,akharinKhabarnews())
    except Exception as e:
        print(f"Error processing element: {e}")
        continue
        
    print(f"Website {url} fully loaded.")
else:
    # Save sentences to files
    save_sentences_to_files(result_map)    

# Close the browser
print("All websites processed.")

Loading website 1/3: https://nournews.ir/fa/news/206863/%D8%AF%D8%B1%DA%AF%DB%8C%D8%B1%DB%8C-%D8%A7%D8%B1%D8%AA%D8%B4-%D9%84%D8%A8%D9%86%D8%A7%D9%86-%D8%A8%D8%A7-%DA%AF%D8%B1%D9%88%D9%87-%D9%87%D8%A7%DB%8C-%D9%85%D8%B3%D9%84%D8%AD-%D8%B3%D9%88%D8%B1%DB%8C
public:https://nournews.ir
Error processing element: name 'result' is not defined
Loading website 2/3: https://nournews.ir/fa/news/199962/%D9%88%D8%B8%DB%8C%D9%81%D9%87-%D9%87%D9%85%D9%87-%DA%A9%D8%B4%D9%88%D8%B1%D9%87%D8%A7%DB%8C-%D8%B5%D9%84%D8%AD-%D8%AF%D9%88%D8%B3%D8%AA%D8%8C-%D8%AA%D9%84%D8%A7%D8%B4-%D8%A8%D8%B1%D8%A7%DB%8C-%D8%A8%D8%B1%D9%82%D8%B1%D8%A7%D8%B1%DB%8C-%D8%A2%D8%AA%D8%B4%E2%80%8C%D8%A8%D8%B3-%D8%AF%D8%B1-%D9%84%D8%A8%D9%86%D8%A7%D9%86-%D9%88-%D8%BA%D8%B2%D9%87-%D8%A7%D8%B3%D8%AA
public:https://nournews.ir
Error processing element: name 'result' is not defined
Loading website 3/3: https://nournews.ir/fa/news/204738/%D9%88%D8%A7%DA%A9%D9%86%D8%B4-%D9%88%D8%B2%D8%A7%D8%B1%D8%AA-%D8%AE%D8%A7%D8%B1%D8%AC%D9%87-%D8%A8%D9%

In [55]:
# # Sort the dictionary based on total_count
# sorted_result_map = dict(sorted(result_map.items(), key=lambda item: item[1][0], reverse=True))

# # Print the sorted result
# for key, value in sorted_result_map.items():
#     print(f"{key}: {value}")

In [56]:
for key, sentence in result_map.items():
    print(f"Sentence for {key}: {sentence}")

Sentence for 0: درگیری ارتش لبنان با گروه های مسلح سوری
ارتش لبنان روز جمعه در بیانیه‌ای از وقوع درگیری در مرز با سوریه خبر داد و اعلام کرد که در جریان آن یک سرباز لبنانی زخمی شده است.
نورنیوز-گروه بین الملل: ارتش لبنان در این بیانیه اعلام کرد: یک یگان ارتش لبنان برای بستن یک گذرگاه غیرقانونی در مرز لبنان با سوریه در منطقه معربون- بعلبلک با نیروهای سوری درگیر شد.
براساس این گزارش، نیروهای ارتش لبنان در این درگیری اقدام به تیراندازی هوایی کردند که در پاسخ، گروه‌های سوری به سمت نیروهای ارتش لبنان آتش گشودند و در نتیجه یکی از سربازان ارتش لبنان زخمی شد.
این درگیری منجر به افزایش تدابیر نظامی توسط یگان‌های مستقر در منطقه شد.
همچنین خبرنگار النشره گزارش داد که ارتش لبنان نیروهای کمکی به محل درگیری در مرز لبنان با سوریه اعزام کرده است.

نورنیوز
Sentence for 2: واکنش وزارت خارجه به برخی ادعاها درباره نحوه خروج ایرانیان از سوریه و اظهارات پوتین
سخنگوی وزارت امور خارجه گفت: حضور مستشاری ایران در سوریه از ابتدا با هدف کمک به ارتش این کشور و برای مقابله با تروریسم و جلوگیری از سرایت ناامنی به کل 

In [57]:

# Iterate through the result_map
for key, value in result_map.items():
    sentence = value[1]
    paragraph = doc.add_paragraph(sentence)

    # Set font size
    run = paragraph.runs[0]
    run.font.size = Pt(14)

    # Set alignment to Right-to-Left and right alignment
    paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT
    paragraph_format = paragraph.paragraph_format
    paragraph_format.space_after = Pt(14)  # Add space after each paragraph

    # Set RTL direction
    paragraph_element = paragraph._p
    paragraph_properties = paragraph_element.get_or_add_pPr()
    bidi = OxmlElement('w:bidi')
    bidi.set(qn('w:val'), '1')
    paragraph_properties.append(bidi)

# Save the document
doc.save("news_doc.docx")